In [48]:
from nba_api.stats.endpoints import commonplayerinfo, playerdashboardbyyearoveryear, commonteamroster
from nba_api.stats.static import players, teams
import time
import pandas as pd
import numpy as np

In [49]:
len(players.get_players())

4374

In [50]:
def get_roster_dfs():
    """This method returns a dataframe containing the roster for each team in the league """
    rosters = []
    for team in teams.get_teams():
        time.sleep(1)
        team_roster = commonteamroster.CommonTeamRoster(team_id=team['id'])
        roster = team_roster.get_data_frames()[0]
        rosters.append(roster)
    return rosters


def transform_player_stats_avg(player_stats):
    """This method tailors the player stats to a format that we actually want"""
    player_yby = player_stats.groupby(['GROUP_VALUE']).agg({'GP': np.sum, 'MIN': np.sum, 'FGM': np.sum, 'FGA': np.sum, 
                                                            'FG_PCT': np.mean, 'FG3M': np.sum, 'FG3A': np.sum, 
                                                            'FG3_PCT': np.mean, 'FTM': np.sum, 'FTA': np.sum, 
                                                            'FT_PCT': np.mean, 'OREB': np.sum, 'DREB': np.sum, 
                                                            'REB': np.sum, 'AST': np.sum, 'TOV': np.sum, 'STL': np.sum,
                                                            'BLK': np.sum, 'PTS': np.sum, 'PLUS_MINUS': np.sum
                                                           }).reset_index()   #[['FG3_PCT']].mean().reset_index()
# player_yby = player_stats.by_year_player_dashboard.get_data_frame()[['GROUP_SET', 'GROUP_VALUE', 'TEAM_ID', 'GP', 
#                                                                      'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 
#                                                                      'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 
#                                                                      'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 
#                                                                      'PTS', 'PLUS_MINUS']]

#     player_yby = player_stats.groupby(['GROUP_VALUE']).agg({'GP': np.sum, 'MIN': np.sum, 'FGM': np.sum, 'FGA': np.sum, 
#                                                             'FG3M': np.sum, 'FG3A': np.sum, 
#                                                             'FTM': np.sum, 'FTA': np.sum}).reset_index()
    player_yby = player_yby.sort_values(by=['GROUP_VALUE'], ascending=True).reset_index(drop=True)
    player_yby['experience'] = player_yby.index
#     full_df = player_yby[player_yby.FG3A > 50]

    
    return player_yby

def get_player_stats_by_roster(roster):
    """This method gets all of the year by year player stats for a given roster"""
    
    player_stat_dfs = []
    for index, row in roster.iterrows():
        player_id = row['PLAYER_ID']
        player_name = row['PLAYER']
        time.sleep(.5)
        player_stats = playerdashboardbyyearoveryear.PlayerDashboardByYearOverYear(player_id)
        player_yby = player_stats.by_year_player_dashboard.get_data_frame()[['GROUP_SET', 'GROUP_VALUE', 'TEAM_ID', 'GP', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'PTS', 'PLUS_MINUS']]
        try:
            temp = player_yby
            player_yby = transform_player_stats_avg(player_yby)
        except Exception as e:
            print("Something went wrong?")
            print(temp.to_string())
        player_yby['PLAYER_NAME'] = player_name
        player_yby['PLAYER_ID'] = player_id
        player_yby['age'] = row['AGE']
        player_stat_dfs.append(player_yby)
#         player_transformed = transform_player_stats_avg(player_yby)
#         player_transformed['PLAYER_NAME'] = player_name
#         player_transformed['PLAYER_ID'] = player_id
#         player_transformed['age'] = row['AGE']
#         player_stat_dfs.append(player_transformed)
    return player_stat_dfs

In [51]:
rosters = get_roster_dfs() # Here I obtain a list of players on each team (there may have been a simpler way to do this with another endpoint)

In [52]:
rosters[0]['TeamID'].iloc[0]

1610612737

In [53]:
all_p_df = []
for r in rosters:
    print(r['TeamID'].iloc[0])
    stats = get_player_stats_by_roster(r)
    all_p_df.extend(stats)

1610612737
1610612738
1610612739
Something went wrong?
Empty DataFrame
Columns: [GROUP_SET, GROUP_VALUE, TEAM_ID, GP, MIN, FGM, FGA, FG_PCT, FG3M, FG3A, FG3_PCT, FTM, FTA, FT_PCT, OREB, DREB, REB, AST, TOV, STL, BLK, PTS, PLUS_MINUS]
Index: []
1610612740
Something went wrong?
Empty DataFrame
Columns: [GROUP_SET, GROUP_VALUE, TEAM_ID, GP, MIN, FGM, FGA, FG_PCT, FG3M, FG3A, FG3_PCT, FTM, FTA, FT_PCT, OREB, DREB, REB, AST, TOV, STL, BLK, PTS, PLUS_MINUS]
Index: []
1610612741
Something went wrong?
Empty DataFrame
Columns: [GROUP_SET, GROUP_VALUE, TEAM_ID, GP, MIN, FGM, FGA, FG_PCT, FG3M, FG3A, FG3_PCT, FTM, FTA, FT_PCT, OREB, DREB, REB, AST, TOV, STL, BLK, PTS, PLUS_MINUS]
Index: []
1610612742
Something went wrong?
Empty DataFrame
Columns: [GROUP_SET, GROUP_VALUE, TEAM_ID, GP, MIN, FGM, FGA, FG_PCT, FG3M, FG3A, FG3_PCT, FTM, FTA, FT_PCT, OREB, DREB, REB, AST, TOV, STL, BLK, PTS, PLUS_MINUS]
Index: []
1610612743
Something went wrong?
Empty DataFrame
Columns: [GROUP_SET, GROUP_VALUE, TEAM_ID

In [55]:
all_p_df[380]

,GROUP_VALUE,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,...,AST,TOV,STL,BLK,PTS,PLUS_MINUS,experience,PLAYER_NAME,PLAYER_ID,age
0,2008-09,82,2668.035000,436,1095,0.398,35,129,0.271,349,...,435,274,110,16,1256,-273,0,Russell Westbrook,201566,30
1,2009-10,82,2812.583333,485,1160,0.418,23,104,0.221,329,...,652,268,108,34,1322,277,1,Russell Westbrook,201566,30
2,2010-11,82,2847.380000,614,1390,0.442,34,103,0.330,531,...,670,316,155,30,1793,113,2,Russell Westbrook,201566,30
3,2011-12,66,2331.265000,578,1266,0.457,62,196,0.316,340,...,362,239,112,21,1558,368,3,Russell Westbrook,201566,30
4,2012-13,82,2860.980000,673,1535,0.438,97,300,0.323,460,...,607,273,145,24,1903,630,4,Russell Westbrook,201566,30
5,2013-14,46,1411.703333,346,791,0.437,68,214,0.318,242,...,319,177,88,7,1002,221,5,Russell Westbrook,201566,30
6,2014-15,67,2301.823333,627,1471,0.426,86,288,0.299,546,...,574,293,140,14,1886,247,6,Russell Westbrook,201566,30
7,2015-16,80,2749.468333,656,1444,0.454,101,341,0.296,465,...,834,342,163,20,1878,634,7,Russell Westbrook,201566,30
8,2016-17,81,2801.748333,824,1941,0.425,200,583,0.343,710,...,840,438,132,31,2558,249,8,Russell Westbrook,201566,30
9,2017-18,80,2913.656667,757,1687,0.449,97,326,0.298,417,...,820,381,147,20,2028,384,9,Russell Westbrook,201566,30


In [56]:
valid_players = [player for player in all_p_df if not player.empty]
valid_players[0]

,GROUP_VALUE,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,...,AST,TOV,STL,BLK,PTS,PLUS_MINUS,experience,PLAYER_NAME,PLAYER_ID,age
0,2015-16,55,646.876667,71,175,0.406000,22,83,0.265,44,...,29,24,19,25,208,7,0,Justin Anderson,1626147,24
1,2016-17,150,2454.466667,372,878,0.429333,116,388,0.298,204,...,132,132,82,46,1064,-388,1,Justin Anderson,1626147,24
2,2017-18,38,519.150000,87,202,0.431000,34,103,0.330,28,...,25,16,15,7,236,-82,2,Justin Anderson,1626147,24


In [57]:
full_df = pd.concat(valid_players)

In [58]:
full_df.head(20)

,GROUP_VALUE,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,...,AST,TOV,STL,BLK,PTS,PLUS_MINUS,experience,PLAYER_NAME,PLAYER_ID,age
0,2015-16,55,646.876667,71,175,0.406000,22,83,0.265,44,...,29,24,19,25,208,7,0,Justin Anderson,1626147,24
1,2016-17,150,2454.466667,372,878,0.429333,116,388,0.298,204,...,132,132,82,46,1064,-388,1,Justin Anderson,1626147,24
2,2017-18,38,519.150000,87,202,0.431000,34,103,0.330,28,...,25,16,15,7,236,-82,2,Justin Anderson,1626147,24
0,2017-18,56,974.210000,144,382,0.377000,72,199,0.362,45,...,79,37,17,5,405,-149,0,Tyler Dorsey,1628416,22
1,2018-19,7,68.898333,10,27,0.370000,2,9,0.222,4,...,5,0,1,0,26,-3,1,Tyler Dorsey,1628416,22
0,2018-19,13,227.350000,22,59,0.373000,11,31,0.355,7,...,20,9,8,3,62,7,0,Kevin Huerter,1628989,20
0,2017-18,6,27.728333,5,11,0.455000,2,5,0.400,0,...,8,2,1,0,12,-12,0,Daniel Hamilton,1627772,23
0,2018-19,13,260.753333,33,81,0.407000,13,37,0.351,10,...,15,9,6,7,89,-53,0,Omari Spellman,1629016,21
0,2010-11,29,284.513333,28,72,0.389000,1,5,0.200,19,...,42,18,33,9,76,8,0,Jeremy Lin,202391,30
1,2011-12,35,940.431667,171,383,0.446000,24,75,0.320,146,...,216,126,55,9,512,142,1,Jeremy Lin,202391,30


In [14]:
sts.to_csv('all_player_data.csv')

In [59]:
def add_adj_age(df):
    df['yr'] = df.GROUP_VALUE.str[:4]
    df['yr'] = df['yr'].astype(int)
    df['adj_age'] = df['age'] - (2018 - df['yr'])
    return df

In [60]:
full_df_age = add_adj_age(full_df)
full_df_age

,GROUP_VALUE,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,...,STL,BLK,PTS,PLUS_MINUS,experience,PLAYER_NAME,PLAYER_ID,age,yr,adj_age
0,2015-16,55,646.876667,71,175,0.406000,22,83,0.265,44,...,19,25,208,7,0,Justin Anderson,1626147,24,2015,21
1,2016-17,150,2454.466667,372,878,0.429333,116,388,0.298,204,...,82,46,1064,-388,1,Justin Anderson,1626147,24,2016,22
2,2017-18,38,519.150000,87,202,0.431000,34,103,0.330,28,...,15,7,236,-82,2,Justin Anderson,1626147,24,2017,23
0,2017-18,56,974.210000,144,382,0.377000,72,199,0.362,45,...,17,5,405,-149,0,Tyler Dorsey,1628416,22,2017,21
1,2018-19,7,68.898333,10,27,0.370000,2,9,0.222,4,...,1,0,26,-3,1,Tyler Dorsey,1628416,22,2018,22
0,2018-19,13,227.350000,22,59,0.373000,11,31,0.355,7,...,8,3,62,7,0,Kevin Huerter,1628989,20,2018,20
0,2017-18,6,27.728333,5,11,0.455000,2,5,0.400,0,...,1,0,12,-12,0,Daniel Hamilton,1627772,23,2017,22
0,2018-19,13,260.753333,33,81,0.407000,13,37,0.351,10,...,6,7,89,-53,0,Omari Spellman,1629016,21,2018,21
0,2010-11,29,284.513333,28,72,0.389000,1,5,0.200,19,...,33,9,76,8,0,Jeremy Lin,202391,30,2010,22
1,2011-12,35,940.431667,171,383,0.446000,24,75,0.320,146,...,55,9,512,142,1,Jeremy Lin,202391,30,2011,23


In [31]:
# full_df_age["3PCT"] = full_df_age["FG3M"] / full_df_age["FG3A"]
# full_df_age["FTPCT"] = full_df_age["FTM"] / full_df_age["FTA"]
# full_df_age["FGPCT"] = full_df_age["FGM"] / full_df_age["FGA"]
# full_df_age

,GROUP_VALUE,GP,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,experience,PLAYER_NAME,PLAYER_ID,age,yr,adj_age,3PCT,FTPCT,FGPCT
0,2015-16,55,646.876667,71,175,22,83,44,55,0,Justin Anderson,1626147,24,2015,21,0.265060,0.800000,0.405714
1,2016-17,150,2454.466667,372,878,116,388,204,258,1,Justin Anderson,1626147,24,2016,22,0.298969,0.790698,0.423690
2,2017-18,38,519.150000,87,202,34,103,28,38,2,Justin Anderson,1626147,24,2017,23,0.330097,0.736842,0.430693
0,2017-18,56,974.210000,144,382,72,199,45,63,0,Tyler Dorsey,1628416,22,2017,21,0.361809,0.714286,0.376963
1,2018-19,7,68.898333,10,27,2,9,4,10,1,Tyler Dorsey,1628416,22,2018,22,0.222222,0.400000,0.370370
0,2018-19,13,227.350000,22,59,11,31,7,8,0,Kevin Huerter,1628989,20,2018,20,0.354839,0.875000,0.372881
0,2017-18,6,27.728333,5,11,2,5,0,0,0,Daniel Hamilton,1627772,23,2017,22,0.400000,NaN,0.454545
0,2018-19,13,260.753333,33,81,13,37,10,14,0,Omari Spellman,1629016,21,2018,21,0.351351,0.714286,0.407407
0,2010-11,29,284.513333,28,72,1,5,19,25,0,Jeremy Lin,202391,30,2010,22,0.200000,0.760000,0.388889
1,2011-12,35,940.431667,171,383,24,75,146,183,1,Jeremy Lin,202391,30,2011,23,0.320000,0.797814,0.446475


In [32]:
full_df_age.to_csv("shooting_year_by_year.csv")

In [61]:
full_df_age# full_df_age

,GROUP_VALUE,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,...,STL,BLK,PTS,PLUS_MINUS,experience,PLAYER_NAME,PLAYER_ID,age,yr,adj_age
0,2015-16,55,646.876667,71,175,0.406000,22,83,0.265,44,...,19,25,208,7,0,Justin Anderson,1626147,24,2015,21
1,2016-17,150,2454.466667,372,878,0.429333,116,388,0.298,204,...,82,46,1064,-388,1,Justin Anderson,1626147,24,2016,22
2,2017-18,38,519.150000,87,202,0.431000,34,103,0.330,28,...,15,7,236,-82,2,Justin Anderson,1626147,24,2017,23
0,2017-18,56,974.210000,144,382,0.377000,72,199,0.362,45,...,17,5,405,-149,0,Tyler Dorsey,1628416,22,2017,21
1,2018-19,7,68.898333,10,27,0.370000,2,9,0.222,4,...,1,0,26,-3,1,Tyler Dorsey,1628416,22,2018,22
0,2018-19,13,227.350000,22,59,0.373000,11,31,0.355,7,...,8,3,62,7,0,Kevin Huerter,1628989,20,2018,20
0,2017-18,6,27.728333,5,11,0.455000,2,5,0.400,0,...,1,0,12,-12,0,Daniel Hamilton,1627772,23,2017,22
0,2018-19,13,260.753333,33,81,0.407000,13,37,0.351,10,...,6,7,89,-53,0,Omari Spellman,1629016,21,2018,21
0,2010-11,29,284.513333,28,72,0.389000,1,5,0.200,19,...,33,9,76,8,0,Jeremy Lin,202391,30,2010,22
1,2011-12,35,940.431667,171,383,0.446000,24,75,0.320,146,...,55,9,512,142,1,Jeremy Lin,202391,30,2011,23


In [33]:
temp = full_df_age
temp['max_experience'] = temp.groupby(['PLAYER_ID'])['experience'].max()
temp

,GROUP_VALUE,GP,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,experience,PLAYER_NAME,PLAYER_ID,age,yr,adj_age,3PCT,FTPCT,FGPCT,max_experience
0,2015-16,55,646.876667,71,175,22,83,44,55,0,Justin Anderson,1626147,24,2015,21,0.265060,0.800000,0.405714,NaN
1,2016-17,150,2454.466667,372,878,116,388,204,258,1,Justin Anderson,1626147,24,2016,22,0.298969,0.790698,0.423690,NaN
2,2017-18,38,519.150000,87,202,34,103,28,38,2,Justin Anderson,1626147,24,2017,23,0.330097,0.736842,0.430693,NaN
0,2017-18,56,974.210000,144,382,72,199,45,63,0,Tyler Dorsey,1628416,22,2017,21,0.361809,0.714286,0.376963,NaN
1,2018-19,7,68.898333,10,27,2,9,4,10,1,Tyler Dorsey,1628416,22,2018,22,0.222222,0.400000,0.370370,NaN
0,2018-19,13,227.350000,22,59,11,31,7,8,0,Kevin Huerter,1628989,20,2018,20,0.354839,0.875000,0.372881,NaN
0,2017-18,6,27.728333,5,11,2,5,0,0,0,Daniel Hamilton,1627772,23,2017,22,0.400000,NaN,0.454545,NaN
0,2018-19,13,260.753333,33,81,13,37,10,14,0,Omari Spellman,1629016,21,2018,21,0.351351,0.714286,0.407407,NaN
0,2010-11,29,284.513333,28,72,1,5,19,25,0,Jeremy Lin,202391,30,2010,22,0.200000,0.760000,0.388889,NaN
1,2011-12,35,940.431667,171,383,24,75,146,183,1,Jeremy Lin,202391,30,2011,23,0.320000,0.797814,0.446475,NaN


In [62]:
max_exp = full_df_age.groupby(['PLAYER_ID'], sort=False)['experience'].max().reset_index()
max_exp.rename(columns={'experience': 'max_experience'}, inplace=True)
max_exp

,PLAYER_ID,max_experience
0,1626147,2
1,1628416,1
2,1628989,0
3,1627772,0
4,1629016,0
5,202391,8
6,1629121,0
7,1629027,0
8,1627752,2
9,203473,5


In [63]:
max_mrg = pd.merge(full_df_age, max_exp, on='PLAYER_ID', how='outer')
max_mrg


,GROUP_VALUE,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,...,BLK,PTS,PLUS_MINUS,experience,PLAYER_NAME,PLAYER_ID,age,yr,adj_age,max_experience
0,2015-16,55,646.876667,71,175,0.406000,22,83,0.265,44,...,25,208,7,0,Justin Anderson,1626147,24,2015,21,2
1,2016-17,150,2454.466667,372,878,0.429333,116,388,0.298,204,...,46,1064,-388,1,Justin Anderson,1626147,24,2016,22,2
2,2017-18,38,519.150000,87,202,0.431000,34,103,0.330,28,...,7,236,-82,2,Justin Anderson,1626147,24,2017,23,2
3,2017-18,56,974.210000,144,382,0.377000,72,199,0.362,45,...,5,405,-149,0,Tyler Dorsey,1628416,22,2017,21,1
4,2018-19,7,68.898333,10,27,0.370000,2,9,0.222,4,...,0,26,-3,1,Tyler Dorsey,1628416,22,2018,22,1
5,2018-19,13,227.350000,22,59,0.373000,11,31,0.355,7,...,3,62,7,0,Kevin Huerter,1628989,20,2018,20,0
6,2017-18,6,27.728333,5,11,0.455000,2,5,0.400,0,...,0,12,-12,0,Daniel Hamilton,1627772,23,2017,22,0
7,2018-19,13,260.753333,33,81,0.407000,13,37,0.351,10,...,7,89,-53,0,Omari Spellman,1629016,21,2018,21,0
8,2010-11,29,284.513333,28,72,0.389000,1,5,0.200,19,...,9,76,8,0,Jeremy Lin,202391,30,2010,22,8
9,2011-12,35,940.431667,171,383,0.446000,24,75,0.320,146,...,9,512,142,1,Jeremy Lin,202391,30,2011,23,8


In [64]:
max_mrg.to_csv("total_player_stats_by_year.csv")